# Sistema de recomendación de anime

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd 
from sklearn.neighbors import NearestNeighbors

#Barra de progreso
import time
from tqdm import tqdm


El objetivo es aplicar todo lo que hemos aprendido hasta el momento, especialmente sobre gradiente descendiente. Para esto desarrollaremos un sistema de recomendación usando factorización de matrices.

En esta entrega se darán todos elementos necesarios para desarrollar un sistema de recomendación de amines, el objetivo es que usted entienda como funciona y desarrolle una solución.

Si desea conocer mas sobre el dataset que usaremos puede revisar acá: https://www.kaggle.com/CooperUnion/anime-recommendations-database

1- Cargue las bases de datos de calificaciones, usuarios y animes. Puede usar para este objetivo la biblioteca pandas. Genere el conjunto de entrenamiento usando los usuarios cómo filas y los animes cómo columnas. Tenga en cuenta que para el algoritmo es necesario que los -1 sean cero, sin embargo, para la recomendación si debe tener en cuenta dichos valores.

In [2]:
#Obtener Listado .CSV
def cargarAnimes():
    df = pd.read_csv('anime.csv', delimiter=',', parse_dates=[6], header=0,index_col=False, squeeze=False)
    #dfAux=pd.DataFrame(df).astype({"anime_id": float, "name": str,"genre": str,"type": str,"episodes": str,"rating": str,"members": str})     
    d=np.array(df)
    dtype=[("anime_id",np.int32),("name",'object'),("genre",'object'),("type",'object'),("episodes",'object'),("rating",'object'),("members",'object')]
    values=[0 for i in range(len(d))]
    for i in range(len(d)):
        values[i]=tuple(d[i])
    aux=np.array(values,dtype=dtype)
    aux=np.sort(aux,order='anime_id')
    values1=[0 for i in range(len(aux))]
    for i in range(len(aux)):
        values1[i]=list(aux[i])
    return np.array(values1)

#Números globales cantidad animes y usuarios
cantAnimes=12294
cantUsers=73516

#Crear R a partir del archivo csv
def crearR(ceros):
    aux=-1
    if ceros:
        aux=0
    uf = pd.read_csv('rating.csv', delimiter=',', parse_dates=[2], header=0,index_col=False, squeeze=False)
    u=np.array(uf)
    
    #Crear R
    R = np.zeros((cantUsers,cantAnimes), dtype = np.int8)
    for i in u:
        if i[1] <= cantAnimes and int(i[2])>=aux:
            R[i[0]-1,i[1]-1] = int(i[2])
      
    return R 


2- Agregue la función donde se ejecuta el algoritmo de factorización de matrices. Puede basarse en el algoritmo compartido en las dispositivas. No debe de gastar todas las iteraciones. 

In [3]:
def matrix_factorization(R, P, Q, K, steps=500, alpha=0.01, beta=0.001):
    Q = Q.T
    error=0
    for step in tqdm(range(steps)):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        aux1=(2 * eij * Q[k][j] - beta * P[i][k])
                        aux2=(2 * eij * P[i][k] - beta * Q[k][j])
                        P[i][k] = P[i][k] + alpha * aux1
                        Q[k][j] = Q[k][j] + alpha * aux2
        eR = np.dot(P,Q)
        e = 0
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
        error=e
        if e < 0.5:
            break
    print('Error',error)
    return P, Q.T

3- Programe una función para generar $\hat{R}$. Recuerde que para generar $\hat{R}$ debe realizar los siguiente pasos:

    - Generar R usando el algoritmo de vecinos más cercanos.
    - Defina K.
    - Defina P y Q.
    - Generar R prima.


In [4]:
def guardar(name,matriz):
    np.savetxt(name,matriz)

def cargar(name):
    return np.loadtxt(name)

def generateRandomPQ(k):
    Q = np.random.random((cantAnimes, k)).astype(np.float32)
    P = np.random.random((cantUsers, k)).astype(np.float32)
    return P,Q

def generateR(usuario, vecinos):
    R = crearR(True)
    RSelected = np.empty([vecinos,cantAnimes])
    distances,indices=knn(True,usuario,vecinos,R)
    for i in range(0, len(indices[0])-1, 1):
        RSelected[i] = R[indices[0,i]]
    del R
    return RSelected

def knn(cargado,usuario,vecinos,R):
    if not(cargado):
        nbrs = NearestNeighbors(n_neighbors=vecinos, algorithm='auto', n_jobs=-1).fit(R)
        distances, indices = nbrs.kneighbors(R[usuario-1].reshape((1, -1)))
        return distances,indices
    else:
        if vecinos>30:
            vecinos=30
        if usuario in [51,510,17000,27005,37502,57502,67501]:
            distances=np.loadtxt('distances'+str(usuario)+'.txt')
            indices=np.loadtxt('indices'+str(usuario)+'.txt')
            return distances[:vecinos].reshape(1,vecinos),indices[:vecinos].reshape(1,vecinos).astype(int)
        else:
            distances=np.loadtxt('distances.txt')
            indices=np.loadtxt('indices.txt')
            return distances.reshape(1,vecinos),indices.reshape(1,vecinos).astype(int)
        
def optimizarMatriz(R,indices,distances):
    userIguales=np.array([]).reshape(0,2)
    cont=0
    pos=-1
    Raux=np.zeros((len(R),len(R[0])),dtype=np.int8)
    for i in tqdm(range(len(indices))):
        for j in range(len(indices[0])):
            if int(distances[i,j]) == 0:
                index=int(indices[i,j])
                aux1=np.where(userIguales[:,1]==i)
                if i != index:
                    aux2=np.where(userIguales[:,1]==index)
                    if len(aux1[0]) == 0:
                        if len(aux2[0]) == 0:
                            pos=pos+1
                            userIguales=np.insert(userIguales, userIguales.shape[0], np.array([pos,i]), 0)
                            userIguales=np.insert(userIguales, userIguales.shape[0], np.array([pos,index]), 0)
                            Raux[cont]=R[i]
                            cont=cont+1
                        else:
                            userIguales=np.insert(userIguales,aux2[0][-1]+1, np.array([userIguales[aux2[0][-1],0],i]), 0)
                    else:
                        if len(aux2[0]) == 0:
                            userIguales=np.insert(userIguales,aux1[0][-1]+1, np.array([userIguales[aux1[0][-1],0],index]), 0)                  
                else:
                    if len(aux1[0]) == 0:
                        pos=pos+1
                        userIguales=np.insert(userIguales, userIguales.shape[0], np.array([pos,i]), 0)
                        Raux[cont]=R[i]
                        cont=cont+1
    return Raux[:cont,:], userIguales

def generateRPrime(k, usuario, vecinos):
    P,Q = generateRandomPQ(k)
    RSelected = generateR(usuario, vecinos)
    P,Q = matrix_factorization(RSelected, P, Q, k)
    return RSelected, np.dot(P, Q.T)

def obtenerRPrima(cargado,usuario,kNeighbors,kFactorizacion):
    if not(cargado) or not(usuario in [51,510,17000,27005,37502,57502,67501]):
        Rl, Rp = generateRPrime(kFactorizacion, usuario, kNeighbors)
        return Rl,Rp
    else:
        Rl=crearR(False)
        Rp=cargar('Rp'+str(usuario)+'V20.txt').reshape(12294,1).T
        return Rl,Rp

4- Realice las siguiente recomendaciones (animes que no ha visto con calificación 10):

    1- Usuario de id 51, películas.
    2- Usuario de id 510, animes con menos de 31 episodios.
    3- Usuario de id 17000, animes con rankins mayores o iguales a 9.
    4- Usuario de id 27005, animes de comedia y aventura.
    5- Usuario de id 37502, TV.
    6- Usuario de id 57502, películas y OVAS.
    7- Usuario de id 67501.

Cada recomendación debe tener como respuesta, número de iteraciones, número de recomendaciones y nombre de los animes,

In [16]:
def recomendar(RpSelected,RSelected,d,idUser,minRating=10,genre=[],tipo=[],episodios=None,ranking=None,members=None):  
    recomendacion=np.array([],dtype='object').reshape(0,len(d[0]))
    for i in tqdm(range(len(d))):
        if RpSelected[i]>=minRating and RSelected[0,i]!=-1:
            boolean=True
            m = np.where(d[:,0] == str(i))[0]
            if genre!=None and len(genre)!=0:
                ward=False
                for j in genre:
                    if j in d[m,2]:
                        ward=True
                if not(ward):
                    boolean=False  
            if boolean and len(tipo)!=0:
                ward=False
                try:
                    for j in tipo:
                        if j in d[m,3]:
                            ward=True
                    if not(ward):
                        boolean=False
                except TypeError or ValueError:
                        boolean=False
            if boolean and episodios!=None:
                try:
                    if not(episodios >= int(d[m,4])):
                        boolean=False
                except TypeError or ValueError:
                        boolean=False
            if boolean and ranking!=None:
                try:
                    if not(ranking <= float(d[m,5])):
                        boolean=False
                except TypeError or ValueError:
                        boolean=False  
            if boolean and members!=None:
                try:
                    if not(members <= int(d[m,5])):
                        boolean=False
                except TypeError or ValueError:
                        boolean=False 
            if boolean:
                recomendacion=np.insert(recomendacion, recomendacion.shape[0], d[m], 0)                
    return recomendacion

def imprimirRecomendacion(recomendacion):
    pd.set_option("display.max_rows", None, "display.max_columns", None)
    rd = {'id' : pd.Series(recomendacion[:,0].T),'nombre' : pd.Series(recomendacion[:,1].T),'Genero' : pd.Series(recomendacion[:,2].T),'tipo' : pd.Series(recomendacion[:,3].T),'episodios' : pd.Series(recomendacion[:,4].T),'rating' : pd.Series(recomendacion[:,5].T),'miembros' : pd.Series(recomendacion[:,6].T)}
    pd.DataFrame(rd)

In [ ]:
def imprimirDiferencias(idUser,Rl,Rp,minR=0,minRp=0,maxR=10,maxRp=100):
    pos=0
    a=idUser-1
    b=0
    for i in range(len(Rl[0])-1):
        if Rl[a,i] != np.around(Rp[b,i]) and (Rl[a,i]>=minR and Rp[b,i]>=minRp and Rl[a,i]<=maxR and Rp[b,i]<=maxRp):
                print('UserAntes',Rl[a,i],'UserDespues',Rp[b,i])
                pos=pos+1
    print('Cantidad Diferencias: ',pos)
    
print(Rp)
imprimirDiferencias(67501,Rl,Rp,minR=1)

In [8]:
#Establecer Parametros

usuario=51
kNeighbors=20
kFactorizacion=20

#Obtener R Prima y Recomendar
Rl,Rp=obtenerRPrima(True,usuario,kNeighbors,kFactorizacion)
recomendacion=recomendar(Rp[0,:],Rl,cargarAnimes(),usuario,minRating=10,tipo=['Movie'])
del Rl
del Rp

pd.set_option("display.max_rows", None, "display.max_columns", None)
rd = {'id' : pd.Series(recomendacion[:,0].T),'nombre' : pd.Series(recomendacion[:,1].T),'Genero' : pd.Series(recomendacion[:,2].T),'tipo' : pd.Series(recomendacion[:,3].T),'episodios' : pd.Series(recomendacion[:,4].T),'rating' : pd.Series(recomendacion[:,5].T),'miembros' : pd.Series(recomendacion[:,6].T)}
aux=pd.DataFrame(rd)
aux

100%|████████████████████████████████████████████████████████████████████████| 12294/12294 [00:00<00:00, 186857.32it/s]


,id,nombre,Genero,tipo,episodios,rating,miembros
0,807,Tsubasa Chronicle: Tori Kago no Kuni no Himegimi,"Action, Adventure, Drama, Fantasy, Shounen",Movie,1,7.39,33633
1,1029,Omoide Poroporo,"Drama, Romance, Slice of Life",Movie,1,7.64,39589
2,1034,Ouritsu Uchuugun: Honneamise no Tsubasa,"Action, Drama, Military, Sci-Fi, Space",Movie,1,7.72,18469
3,1096,Mobile Police Patlabor 2: The Movie,"Drama, Mecha, Mystery, Police, Sci-Fi",Movie,1,7.83,11957
4,1301,Ginga Tetsudou 999 (Movie),"Adventure, Drama, Fantasy, Sci-Fi, Space",Movie,1,7.76,7237
5,1723,Clannad Movie,"Drama, Fantasy, Romance, School",Movie,1,7.35,99506
6,1943,Paprika,"Fantasy, Horror, Mystery, Psychological, Sci-F...",Movie,1,8.15,163308
7,2176,Kiddy Grade: Ignition,"Action, Mecha, Sci-Fi, Super Power",Movie,1,7.21,3466
8,2211,Jumping,Adventure,Movie,1,6.69,2341
9,2312,Daishizen no Majuu: Bagi,"Action, Romance, Sci-Fi",Movie,1,6.55,1479


In [12]:
#Establecer Parametros

usuario=510
kNeighbors=20
kFactorizacion=20

#Obtener R Prima y Recomendar
Rl,Rp=obtenerRPrima(True,usuario,kNeighbors,kFactorizacion)
recomendacion=recomendar(Rp[0,:],Rl,cargarAnimes(),usuario,minRating=10,episodios=31)
del Rl
del Rp

pd.set_option("display.max_rows", None, "display.max_columns", None)
rd = {'id' : pd.Series(recomendacion[:,0].T),'nombre' : pd.Series(recomendacion[:,1].T),'Genero' : pd.Series(recomendacion[:,2].T),'tipo' : pd.Series(recomendacion[:,3].T),'episodios' : pd.Series(recomendacion[:,4].T),'rating' : pd.Series(recomendacion[:,5].T),'miembros' : pd.Series(recomendacion[:,6].T)}
aux=pd.DataFrame(rd)
aux

100%|████████████████████████████████████████████████████████████████████████| 12294/12294 [00:00<00:00, 342409.22it/s]


,id,nombre,Genero,tipo,episodios,rating,miembros
0,25,Sunabouzu,"Action, Adventure, Comedy, Ecchi, Sci-Fi, Shounen",TV,24,7.55,65516
1,163,Power Stone,"Adventure, Comedy, Fantasy, Historical, Shounen",TV,26,6.57,4308
2,286,B&#039;T X,"Adventure, Mecha, Sci-Fi, Shounen",TV,25,7.17,5913
3,544,Venus Senki,"Action, Adventure, Sci-Fi, Space",Movie,1,6.94,5204
4,758,RahXephon: Pluralitas Concentio,"Action, Drama, Mecha, Psychological, Romance, ...",Movie,1,7.16,10524
5,819,Boku wa Imouto ni Koi wo Suru,"Romance, Shoujo",OVA,1,6.57,26923
6,975,Yokohama Kaidashi Kikou,"Sci-Fi, Seinen, Slice of Life",OVA,2,7.37,12389
7,1458,Uchuu no Kishi Tekkaman,"Action, Adventure, Mecha, Sci-Fi, Shounen, Space",TV,26,6.55,1059
8,1656,PostPet Momobin,"Comedy, Kids",TV,24,5.68,215
9,1941,Moonlight Mile 1st Season: Lift Off,"Adventure, Drama, Sci-Fi, Space",TV,12,7.27,6037


In [18]:
#Establecer Parametros

usuario=17000
kNeighbors=20
kFactorizacion=20

#Obtener R Prima y Recomendar
Rl,Rp=obtenerRPrima(True,usuario,kNeighbors,kFactorizacion)
recomendacion=recomendar(Rp[0,:],Rl,cargarAnimes(),usuario,minRating=10)
del Rl
del Rp

pd.set_option("display.max_rows", None, "display.max_columns", None)
rd = {'id' : pd.Series(recomendacion[:,0].T),'nombre' : pd.Series(recomendacion[:,1].T),'Genero' : pd.Series(recomendacion[:,2].T),'tipo' : pd.Series(recomendacion[:,3].T),'episodios' : pd.Series(recomendacion[:,4].T),'rating' : pd.Series(recomendacion[:,5].T),'miembros' : pd.Series(recomendacion[:,6].T)}
aux=pd.DataFrame(rd)
aux

100%|████████████████████████████████████████████████████████████████████████| 12294/12294 [00:00<00:00, 273929.56it/s]


,id,nombre,Genero,tipo,episodios,rating,miembros
0,50,Aa! Megami-sama! (TV),"Comedy, Magic, Romance, Seinen, Supernatural",TV,24,7.45,96694
1,163,Power Stone,"Adventure, Comedy, Fantasy, Historical, Shounen",TV,26,6.57,4308
2,243,Gravitation,"Comedy, Music, Romance, Shoujo, Shounen Ai",TV,13,7.26,67916
3,468,Ghost in the Shell 2: Innocence,"Mecha, Military, Police, Psychological, Sci-Fi",Movie,1,7.93,85714
4,469,Karin,"Comedy, Romance, School, Shounen, Vampire",TV,24,7.33,106142
5,605,Tenshi ni Narumon!,"Comedy, Fantasy, Romance",TV,26,6.91,3847
6,694,Canvas 2: Niji-iro no Sketch,"Comedy, Drama, Romance, Slice of Life",TV,24,6.91,19491
7,731,Interstella5555: The 5tory of The 5ecret 5tar ...,"Adventure, Drama, Music, Sci-Fi",Music,1,8.17,31464
8,800,NieA Under 7,"Comedy, Sci-Fi, Slice of Life",TV,13,6.99,16711
9,819,Boku wa Imouto ni Koi wo Suru,"Romance, Shoujo",OVA,1,6.57,26923


In [19]:
#Establecer Parametros

usuario=27005
kNeighbors=20
kFactorizacion=20

#Obtener R Prima y Recomendar
Rl,Rp=obtenerRPrima(True,usuario,kNeighbors,kFactorizacion)
recomendacion=recomendar(Rp[0,:],Rl,cargarAnimes(),usuario,minRating=10,genre=['Comedy','Adventure'])
del Rl
del Rp

pd.set_option("display.max_rows", None, "display.max_columns", None)
rd = {'id' : pd.Series(recomendacion[:,0].T),'nombre' : pd.Series(recomendacion[:,1].T),'Genero' : pd.Series(recomendacion[:,2].T),'tipo' : pd.Series(recomendacion[:,3].T),'episodios' : pd.Series(recomendacion[:,4].T),'rating' : pd.Series(recomendacion[:,5].T),'miembros' : pd.Series(recomendacion[:,6].T)}
aux=pd.DataFrame(rd)
aux

100%|████████████████████████████████████████████████████████████████████████| 12294/12294 [00:00<00:00, 293496.95it/s]


,id,nombre,Genero,tipo,episodios,rating,miembros
0,3849,The SoulTaker: Tamashii-gari - Komugi Daisakusen,Comedy,Special,3,6.1,1266
1,8632,Chuuzumou,Comedy,Movie,1,6.8,988
2,9070,Oden-kun,Comedy,TV,156,7.36,118


In [20]:
#Establecer Parametros

usuario=37502
kNeighbors=20
kFactorizacion=20

#Obtener R Prima y Recomendar
Rl,Rp=obtenerRPrima(True,usuario,kNeighbors,kFactorizacion)
recomendacion=recomendar(Rp[0,:],Rl,cargarAnimes(),usuario,minRating=10,tipo=['TV'])
del Rl
del Rp

pd.set_option("display.max_rows", None, "display.max_columns", None)
rd = {'id' : pd.Series(recomendacion[:,0].T),'nombre' : pd.Series(recomendacion[:,1].T),'Genero' : pd.Series(recomendacion[:,2].T),'tipo' : pd.Series(recomendacion[:,3].T),'episodios' : pd.Series(recomendacion[:,4].T),'rating' : pd.Series(recomendacion[:,5].T),'miembros' : pd.Series(recomendacion[:,6].T)}
aux=pd.DataFrame(rd)
aux

100%|███████████████████████████████████████████████████████████████████████| 12294/12294 [00:00<00:00, 1117523.59it/s]


,id,nombre,Genero,tipo,episodios,rating,miembros
0,8,Beet the Vandel Buster,"Adventure, Fantasy, Shounen, Supernatural",TV,52,7.06,9848
1,2489,Onegai My Melody,"Comedy, Fantasy, Magic",TV,52,7.16,3633


In [21]:
#Establecer Parametros

usuario=57502
kNeighbors=20
kFactorizacion=20

#Obtener R Prima y Recomendar
Rl,Rp=obtenerRPrima(True,usuario,kNeighbors,kFactorizacion)
recomendacion=recomendar(Rp[0,:],Rl,cargarAnimes(),usuario,minRating=10,tipo=['OVA','Movie'])
del Rl
del Rp

pd.set_option("display.max_rows", None, "display.max_columns", None)
rd = {'id' : pd.Series(recomendacion[:,0].T),'nombre' : pd.Series(recomendacion[:,1].T),'Genero' : pd.Series(recomendacion[:,2].T),'tipo' : pd.Series(recomendacion[:,3].T),'episodios' : pd.Series(recomendacion[:,4].T),'rating' : pd.Series(recomendacion[:,5].T),'miembros' : pd.Series(recomendacion[:,6].T)}
aux=pd.DataFrame(rd)
aux

100%|█████████████████████████████████████████████████████████████████████████| 12294/12294 [00:00<00:00, 67355.63it/s]


,id,nombre,Genero,tipo,episodios,rating,miembros
0,43,Ghost in the Shell,"Action, Mecha, Police, Psychological, Sci-Fi, ...",Movie,1,8.34,223036
1,87,Mobile Suit Gundam: Char&#039;s Counterattack,"Drama, Mecha, Military, Sci-Fi, Space",Movie,1,7.81,19258
2,214,Pia Carrot e Youkoso!! 2,"Comedy, Hentai, Romance",OVA,3,6.39,1408
3,358,"Iriya no Sora, UFO no Natsu","Drama, Romance, Sci-Fi",OVA,6,7.12,18254
4,432,Iria: Zeiram The Animation,"Action, Adventure, Sci-Fi, Space",OVA,6,7.21,10208
5,512,Majo no Takkyuubin,"Adventure, Comedy, Drama, Fantasy, Magic, Romance",Movie,1,8.27,152331
6,547,Wind: A Breath of Heart OVA,"Drama, Romance, School, Supernatural",OVA,3,6.35,2043
7,661,Elf-ban Kakyuusei: Anata dake wo Mitsumete...,"Drama, Harem, Romance, School",OVA,4,6.45,1849
8,724,Enzai,"Hentai, Horror, Yaoi",OVA,2,5.55,13711
9,725,First Kiss Monogatari,"Drama, Romance",OVA,1,5.21,1878


In [22]:
#Establecer Parametros

usuario=67501
kNeighbors=20
kFactorizacion=20

#Obtener R Prima y Recomendar
Rl,Rp=obtenerRPrima(True,usuario,kNeighbors,kFactorizacion)
recomendacion=recomendar(Rp[0,:],Rl,cargarAnimes(),usuario,minRating=10)
del Rl
del Rp

pd.set_option("display.max_rows", None, "display.max_columns", None)
rd = {'id' : pd.Series(recomendacion[:,0].T),'nombre' : pd.Series(recomendacion[:,1].T),'Genero' : pd.Series(recomendacion[:,2].T),'tipo' : pd.Series(recomendacion[:,3].T),'episodios' : pd.Series(recomendacion[:,4].T),'rating' : pd.Series(recomendacion[:,5].T),'miembros' : pd.Series(recomendacion[:,6].T)}
aux=pd.DataFrame(rd)
aux

100%|████████████████████████████████████████████████████████████████████████| 12294/12294 [00:00<00:00, 770923.70it/s]


,id,nombre,Genero,tipo,episodios,rating,miembros
0,1281,Gakkou no Kaidan,"Horror, Mystery, Supernatural",TV,19,7.71,42033
1,1527,Pokemon: Senritsu no Mirage Pokemon,"Adventure, Comedy, Fantasy, Kids",Special,1,6.57,13040
2,1588,Mamoru-kun ni Megami no Shukufuku wo!,"Comedy, Romance, School",TV,24,7.21,16922
3,1853,Cutey Honey F,"Action, Adventure, Comedy, Magic, Romance, Sci...",TV,39,6.72,3495
4,2070,Guy: Youma Kakusei,"Action, Hentai, Sci-Fi, Space",OVA,2,5.84,888
5,2102,Tattoon Master,"Adventure, Comedy, Slice of Life, Supernatural",OVA,2,6.3,1370
6,2387,Ai Monogatari: 9 Love Stories,"Drama, Romance",OVA,1,6.82,2060
7,3166,Ayakashi,"Action, Fantasy, Horror, Sci-Fi",TV,12,6.5,20882
8,3557,Shoujo Auction,Hentai,OVA,2,6.12,1278
9,4129,Mahou no Tenshi Creamy Mami: Curtain Call,"Comedy, Music, Shoujo",Music,1,6.27,298


In [ ]:
print(cargarAnimes())

In [ ]:
l = np.matrix((["12","12","3"],["34","21","4"]))
print(l)       
m= np.where(l[:,2]==str(3))[0]
print(l[m])